In [24]:
import pandas as pd
import numpy as np
import math

In [25]:
num_templates = 10

In [26]:
file = "tpcds.csv"

# read a csv file where the columnas are delimitted by # and load this to a dataframe
# // Generated by WCA for GP
import pandas as pd

# Load the CSV file into a Pandas data frame
df = pd.read_csv(file, delimiter='#')

In [27]:
df.head()

,tables,joins,predicates,db2,actual
0,"web_sales,date_dim,catalog_sales,date_dim,cust...","cd_demo_sk=c.c_current_cdemo_sk,c.c_customer_s...","d_year,=,2002",126.928,265.964
1,"web_sales,date_dim,catalog_sales,date_dim,cust...","cd_demo_sk=c.c_current_cdemo_sk,c.c_customer_s...","d_year,=,2002",129.740,287.028
2,"web_sales,date_dim,catalog_sales,date_dim,cust...","cd_demo_sk=c.c_current_cdemo_sk,c.c_customer_s...","d_year,=,2002",126.928,258.644
3,"web_sales,date_dim,catalog_sales,date_dim,cust...","cd_demo_sk=c.c_current_cdemo_sk,c.c_customer_s...","d_year,=,2002",129.420,269.756
4,"web_sales,date_dim,catalog_sales,date_dim,cust...","cd_demo_sk=c.c_current_cdemo_sk,c.c_customer_s...","d_year,=,2001",129.008,261.952


In [28]:
df.shape

(13654, 5)

# Labeling queries based on memory usage level

In [29]:
df.shape


(13654, 5)

In [30]:
len(np.unique(df['actual']))

6349

In [31]:
# Generated by WCA for GP
df['template'] = pd.qcut(df['actual'], q=num_templates, labels=False, duplicates='drop')

In [32]:
df.shape

(13654, 6)

In [33]:
# generate code to train test split a dataframe

# import train_test_split

from sklearn.model_selection import train_test_split


# Generated by WCA for GP

# Split the dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Save the train and test sets as CSV files
# train_df.to_csv('train_tpcds.csv', index=False, sep='#')
test_df.to_csv('test_tpcds_{}.csv'.format(num_templates), index=False, sep='#')

In [34]:
train_df['template'].value_counts()

template
0    1900
2    1439
4    1142
8    1092
6    1090
9    1079
7    1078
5    1033
3     797
1     273
Name: count, dtype: int64

In [35]:
median_size = int(np.median(train_df['template'].value_counts()))

In [36]:
train_df.shape

(10923, 6)

In [37]:
train_df['template'].unique()

array([5, 4, 8, 0, 1, 2, 6, 7, 9, 3])

In [38]:
# how to create an empty dataframe that has the same columns as an existing dataframe?
# Generated by WCA for GP
# Create an empty DataFrame with the same columns as df
new_df = pd.DataFrame(columns=train_df.columns)

for i in train_df['template'].unique():
    #print('template: {}'.format(i))
    length = train_df[train_df['template'] == i].shape[0]
    #print('length: {}'.format(length))

    if length == median_size:
        new_df = pd.concat([new_df, train_df[train_df['template']==i].copy()], axis=0)
    elif length > median_size:
        new_df = pd.concat([new_df, train_df[train_df['template']==i].sample(median_size)])
    elif length < median_size:
        ratio = math.ceil(median_size / length)
        temp_df = train_df[train_df['template']==i].copy().sample(n=length * ratio, replace=True)
        temp_df = temp_df.sample(median_size)
        new_df = pd.concat([new_df, temp_df])


/var/folders/6f/_thrtvnj5dj0h94ccbgkdb9r0000gn/T/ipykernel_4328/173364556.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df, temp_df])


In [39]:
new_df.shape

(10840, 6)

In [40]:
new_df['template'].value_counts()

template
5    1084
4    1084
8    1084
0    1084
1    1084
2    1084
6    1084
7    1084
9    1084
3    1084
Name: count, dtype: int64

In [41]:
train_df = new_df.reset_index(drop=True)

In [42]:
train_df.head()

,tables,joins,predicates,db2,actual,template
0,"store_sales,date_dim,store,household_demographics",store_sales.ss_sold_date_sk=date_dim.d_date_sk...,"d_dom,between,1 3,hd_buy_potential,=,501-1000,...",72.656,246.140,5
1,"store_sales,date_dim,store,household_demographics",store_sales.ss_sold_date_sk=date_dim.d_date_sk...,"d_dom,between,1 3,hd_buy_potential,=>,10000,hd...",72.656,246.472,5
2,"store_sales,date_dim,store,household_demographics",store_sales.ss_sold_date_sk=date_dim.d_date_sk...,"d_dom,between,1 3,hd_buy_potential,=,1001-5000...",72.656,245.512,5
3,"store_sales,date_dim,store,household_demographics",store_sales.ss_sold_date_sk=date_dim.d_date_sk...,"d_dom,between,1 3,hd_buy_potential,=,1001-5000...",72.656,245.216,5
4,"store_sales,store_returns,catalog_sales,date_d...","d1.d_date_sk=ss_sold_date_sk,ss_item_sk=sr_ite...","d_quarter_name,=,2002Q1",209.584,232.976,5


In [43]:
train_df.dtypes

tables         object
joins          object
predicates     object
db2           float64
actual        float64
template       object
dtype: object

In [44]:
train_df['template'] = train_df['template'].astype('category')

In [45]:
len(train_df['template'].unique())

10

In [46]:
train_df.to_csv('train_tpcds_{}.csv'.format(num_templates), index=False, sep='#')